In [0]:
#| default_exp preprocessing.lesson16_after_lesson

# Lesson16 Scripts updated after lesson
> Reproducing lesson 16 from the notebook which was updated after lesson

In [0]:
#| hide
%load_ext autoreload
%autoreload 2

In [0]:
#| export
from cv_tools.core import *


In [0]:
#| hide
import nbdev; nbdev.nbdev_export('03_preprocessing.lesson_16_after_lesson.ipynb')